In [1]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

## 추천받은 포트폴리오 -> 잔고(detail 테이블)

In [2]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')
price_db = pd.read_pickle('../../data/external/price_db_d.pkl')
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')    

In [3]:
userid = 'A00'
username = '투자자0'
current_date = '2021-02-01'
risk_profile = 2

# c_date: 추천 포트폴리오DB에서 사용자가 입력한 날짜와 가장 가까운 날짜.
c_date = advised_pf.loc[advised_pf.date <= current_date, ['date']].max().date
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]
remaining_cash = 135786

In [4]:
# 보유수량
detail = pd.DataFrame.from_dict(new_units, orient='index').rename(columns={0:'quantity'})
detail.index.name='itemcode'

# 매입가격
temp = pd.DataFrame.from_dict(prices, orient='index').rename(columns={0:'cost_price'}).drop([1], axis=1)
temp.index.name='itemcode'

# 병합
detail = detail.merge(temp, left_index=True, right_index=True, how='inner')

detail['price'] = detail['cost_price']
detail['cost_value'] = detail['quantity']*detail['cost_price']
detail['value'] = detail['quantity']*detail['cost_price']  # 매입가와 평가가격 동일하다고 가정
detail = detail.merge(instruments_m.loc[:, ['itemcode', 'itemname', 'asset_class']], left_on='itemcode', right_on='itemcode', how='left')
detail = detail.reset_index(drop=True)

In [5]:
df_cash = {
    'itemcode': 'C000001',
    'quantity': remaining_cash,
    'cost_price': 1,
    'cost_value': remaining_cash,
    'price':1
    'value': remaining_cash,
    'itemname': '현금',
    'asset_class': 'Cash'
}
df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T

In [6]:
detail = pd.concat([detail, df_cash])

In [7]:
detail['date'] = c_date
detail['date'] = pd.to_datetime(detail['date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y 4:0:00 PM').astype(str)
detail['userid'] = userid
detail['username'] = username
detail['original'] = 'N'
detail['group_by'] = ''

In [8]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,
0,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,


In [9]:
detail['wt'] = detail.value.transform(lambda x: x/x.sum())

In [10]:
detail = detail.reset_index(drop=True)

In [11]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


## 리밸런싱

In [17]:
from datetime import datetime
current_date

'2021-02-01'

In [18]:
end_date = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].max()
dates = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].unique()

In [19]:
dt = datetime.strptime('2021-02-01', '%Y-%m-%d')
dt = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 09:00:00 AM'

In [21]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


In [69]:
dates = advised_pf.loc[(advised_pf.risk_profile == risk_profile) & (
    advised_pf.date > current_date), 'date'].unique()
every5day = dates[::5]

In [70]:
dates

array(['2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
       '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11',
       '2021-02-12', '2021-02-15', '2021-02-16', '2021-02-17',
       '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23',
       '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01',
       '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05',
       '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11',
       '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17',
       '2021-03-18', '2021-03-19'], dtype=object)

In [ ]:
for dt in dates:
    prices_dt = price_db.loc[price_db.date==dt, ['price', 'itemcode']].reset_index(drop=True)
    holding_itemcodes = detail.itemcode.to_list()
    holding_prices = prices_dt[prices_dt.itemcode.isin(holding_itemcodes)]

In [71]:
dt = dates[0]

In [85]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


In [133]:
prices_dt = price_db.loc[price_db.date==dt, ['price', 'itemcode']].reset_index(drop=True)

In [129]:
holdings = detail.itemcode.to_list()
holdings

['A122260', 'A114260', 'A130730', 'A329650', 'A266370', 'C000001']

In [136]:
prices_dt[prices_dt.itemcode.isin(holdings)]

,price,itemcode
8,100895.0,A130730
9,101105.0,A122260
23,22020.0,A266370
24,58125.0,A114260


In [122]:
price_db.loc[itemcode_idx.values, :]

IndexError: Boolean index has wrong length: 30 instead of 24714

In [96]:
price_itemcodes = price_db.loc[price_db.date==dt, 'itemcode']

In [104]:
price_itemcodes

699      A069500
1430     A105190
2161     A102110
2892     A232080
3623     A278540
4354     A229200
5085     A196230
5816     A293180
6547     A130730
7278     A122260
8009     A139260
8740     A251350
9471     A292150
10202    A091160
10933    A091180
11664    A091170
12395    A283580
13126    A245710
13857    A102780
14588    A214980
15319    A157490
16050    A148070
16781    A153130
17512    A266370
18243    A114260
18974    A195930
19705    A273130
20436    A245340
21167    A237370
3483        CALL
Name: itemcode, dtype: object

In [103]:
price_itemcodes[price_itemcodes == True]

Series([], Name: itemcode, dtype: object)

In [81]:
itemcode_idx

Int64Index([  699,  1430,  2161,  2892,  3623,  4354,  5085,  5816,  6547,
             7278,  8009,  8740,  9471, 10202, 10933, 11664, 12395, 13126,
            13857, 14588, 15319, 16050, 16781, 17512, 18243, 18974, 19705,
            20436, 21167,  3483],
           dtype='int64')

In [82]:
price_db.loc[itemcode_idx]

,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
699,2021-02-02,A069500,42775.0000,12520132.0,398242.608758,0.014719,ETF
699,2009-10-22,CALL,0.0200,NaN,NaN,NaN,riskfree
1430,2021-02-02,A105190,43040.0000,151877.0,14745.605441,0.014982,ETF
1430,2012-09-25,CALL,0.0299,NaN,NaN,NaN,riskfree
2161,2021-02-02,A102110,42610.0000,1653014.0,79438.882264,0.015253,ETF
2161,2015-09-14,CALL,0.0149,NaN,NaN,NaN,riskfree
2892,2021-02-02,A232080,15095.0000,480913.0,13740.016176,0.001326,ETF
2892,2018-09-06,CALL,0.0151,NaN,NaN,NaN,riskfree
3623,2021-02-02,A278540,13815.0000,399964.0,15411.345111,0.012382,ETF
4354,2021-02-02,A229200,15015.0000,4947814.0,128837.817498,0.000000,ETF


## 자산군별 바 차트

In [28]:
import plotly.express as px

In [43]:
v = detail.loc[:, ['wt', 'itemname']].groupby(
                    'itemname').sum().sort_values('wt', ascending=False).reset_index()

In [29]:
a = detail.loc[:, ['wt', 'asset_class']].groupby(
                    'asset_class').sum().sort_values('wt', ascending=False).reset_index()

In [50]:
asset_class = ['Fixed Income', 'Alternative', 'Cash', 'Equity']

In [53]:
df_ac = pd.DataFrame(asset_class, columns=['asset_class'])

In [62]:
df_by_asset_class = a.merge(df_ac, left_on='asset_class', right_on='asset_class', how='right').sort_values('wt', ascending=False).reset_index()

In [32]:
a

,asset_class,wt
0,Fixed Income,0.636082
1,Alternative,0.142776
2,Cash,0.135820
3,Equity,0.085322


In [44]:
fig1 = px.pie(v, names = 'itemname', values = 'wt', title = "추천 포트폴리오", color_discrete_sequence=px.colors.qualitative.Set3)
fig1.show()

In [64]:
df_by_asset_class.loc[:, ['asset_class', 'wt']]

,asset_class,wt
0,Fixed Income,0.636082
1,Alternative,0.142776
2,Cash,0.135820
3,Equity,0.085322


In [65]:
fig2 = px.bar(df_by_asset_class, y = 'wt', x = 'asset_class', title = '자산군별 비중',
       labels = {'asset_class': '자산군', 'wt': '비중'},
       orientation = 'v', color = "asset_class", color_continuous_scale = 'darkmint',
        template = 'plotly_dark')
fig2.show()